In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Util import evaluate_curret, get_invest_period
from os.path import join
import os
import time
import sklearn.covariance
import pickle
from collections import namedtuple, defaultdict
from subprocess import Popen
import pandas as pd

In [2]:
#matrix to send Lauritzen computation
T = 500
N = 1000
with open('pastRets/{}_{}_pastRets.pkl'.format(T,N), 'rb') as f:
    pastRets = pickle.load(f)
#np.save('large_data_matrix.npy', pastRets[0])
#np.save('large_cov_matrix.npy', np.cov(pastRets[0].T))
from estimators import MTP2_wrapper
start = time.time()
print(start)
MTP2_wrapper(pastRets[0])
print(time.time()-start)

In [2]:
run_name='KT_table'
T = 1260
N = 200
KT_or_not = True

In [74]:
all_methods = [
    'glasso',
    'MTP2',
    'LS',
    'old_LS',
    'NLS',
    'LRPS',
    'AFM_NL',
    'AFM_LS',
    'equiweight',
    'POET'
]

In [36]:
def get_portfolio_to_period(end_h, all_methods, T, N, KT_or_not, run_name):
    with open('outRets/{}_outRets.pkl'.format(N), 'rb') as f:
        outRets = pickle.load(f)
    method_to_portfolio = defaultdict(list)
    for h in range(end_h):
        print(h, end= ' ')
        for method_name in all_methods:
            if h == 0:
                print('pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name))
            try:
                if method_name == 'equiweight':
                    cov = np.eye(N)
                else:
                    with open('pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name), 'rb') as f:
                        cov = pickle.load(f)
            except Exception as e:
                print(e)
                print("COULD NOT FINISH h={} for method {}".format(h, method_name))
            #print(cov.shape)
            #print(outRets[h].shape)
            curret = evaluate_curret(cov, outRets[h])
            method_to_portfolio[method_name].append(curret)
    return method_to_portfolio

In [46]:
all_methods = ['CLIME', 'NLS', 'AFM_LS', 'POET']
res = {}
for N, T in [(100, 1260)]:
    method_to_portfolio = get_portfolio_to_period(16, all_methods, T, N, False, 'test_run_new')
    res[(N,T)] = method_to_portfolio

0 pickle/1260_100_0_CLIME_False_test_run_new_covEst.pkl
pickle/1260_100_0_NLS_False_test_run_new_covEst.pkl
pickle/1260_100_0_AFM_LS_False_test_run_new_covEst.pkl
pickle/1260_100_0_POET_False_test_run_new_covEst.pkl
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

In [53]:
for (N,T), method_to_portfolio in res.items():
    print("N: {}, T: {}".format(N,T))
    for method, portfolio in method_to_portfolio.items():
        print(method, get_std(portfolio))

N: 100, T: 1260
CLIME 13.252743300174648
NLS 10.718512743136502
AFM_LS 10.860846221830062
POET 10.93952951819718


In [27]:
from Util import get_IR, get_std
for method_name, portfolio in method_to_portfolio.items():
    print(method_name, get_IR(portfolio))

MTP2 (9.786770303853176, 11.802609922505646, 0.829203910669911)
MTP2_cov (10.627872425898833, 11.904656872721702, 0.8927491602258197)


In [47]:
def initialize_df():
    N_t_tuples = [(100,50),(100,200),(200,100),(200,400),(500,250),(500,1000)]
    idx = pd.MultiIndex.from_tuples(N_t_tuples, names=["N\n(number of assets)", "T\n(lookback period)"])
    col = ['MTP2', 'MTP2_cov']
    df = pd.DataFrame('-', idx, col)
    return df

In [48]:
df = initialize_df()

In [49]:
for N, T in [(100,50),(100,200),(200,100),(200,400),(500,250),(500,1000)]:
    for method in ['MTP2', 'MTP2_cov']:
        if (N,T) not in res:
            continue
        portfolio = res[(N,T)][method]
        if len(portfolio) == 0:
            print(N,T, method)
            continue
        std = get_std(portfolio)
        df.at[(N,T), method] = "{0:.3f}".format(std)

In [50]:
df

MTP2 MTP2_cov
N\n(number of assets) T\n(lookback period)              
100                   50                      -        -
                      200                     -        -
200                   100                     -        -
                      400                     -        -
500                   250                     -        -
                      1000                    -        -